In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

%matplotlib inline

3) удаление стоп-слов, а также пороги минимальной и максимальной document frequency;

4) векторизация документов (CountVectorizer vs. TfIdfVectorizer);

5) что-нибудь ещё?

При оценке классификатора обратите внимание на TP и FP.

построить классификатор, оценить и записать результат; в итоге результаты усреднить. 2) поможет ли параметр class prior probability?

In [2]:
path = 'smsspamcollection/SMSSpamCollection'

messages = pd.read_csv(path, sep='\t',
                           names=["label", "message"])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Загрузили, посмотрели. Смотрим на классы

In [3]:
print(messages.groupby('label').describe())

                                                        message
label                                                          
ham   count                                                4825
      unique                                               4516
      top                                Sorry, I'll call later
      freq                                                   30
spam  count                                                 747
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                    4


Выборка очевидно несбалансирована по классам (~6:1). Соответственно, dummy-classifier, всегда присваивающий ярлык "ham", будет иметь accuracy ~86%, а confusion matrix будет выглядеть вот так:

In [4]:
d = {'clf_ham': pd.Series([4825, 747], index=['ham', 'spam']),
    'clf_spam': pd.Series([0, 0], index=['ham', 'spam'])}
pd.DataFrame(d)

,clf_ham,clf_spam
ham,4825,0
spam,747,0


Для задачи определения спама такой классификатор абсолютно бесполезен просто по определению: это классификатор, который никогда не определяет спам

Соотношение 6:1 - вот и отлично, разделим ham на шесть непересекающихся выборок, обучим классификатор с каждым

In [5]:
# токенизируем, лемматизируем, стеммим, всё вот это запихиваем в один датафрейм
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer


def tokenize(text):
    text = text.lower()
    return word_tokenize(text)


msg_tok = messages['message'].map(tokenize)


def get_rid(arr):
    res = [x.strip(string.punctuation) for x in arr if x.strip(string.punctuation) != '']
    return res


msg_tok_no_punct = msg_tok.map(get_rid)


lm = WordNetLemmatizer()
st = LancasterStemmer()


def lemmatize(arr):
    return list(map(lm.lemmatize, arr))


def stem(arr):
    return list(map(st.stem, arr))

msg_df = pd.DataFrame({'tokenized_punct': msg_tok, 'tokenized_no_punct': msg_tok_no_punct, 
                    'punctlem': msg_tok.map(lemmatize), 'punctstem': msg_tok.map(stem), 
                    'nopunctlem': msg_tok_no_punct.map(lemmatize), 'nopunctstem': msg_tok_no_punct.map(stem), 
                    'label': messages['label']})

In [6]:
msg_df.head()

,label,nopunctlem,nopunctstem,punctlem,punctstem,tokenized_no_punct,tokenized_punct
0,ham,"[go, until, jurong, point, crazy, available, o...","[go, until, jurong, point, crazy, avail, on, i...","[go, until, jurong, point, ,, crazy.., availab...","[go, until, jurong, point, ,, crazy.., avail, ...","[go, until, jurong, point, crazy, available, o...","[go, until, jurong, point, ,, crazy.., availab..."
1,ham,"[ok, lar, joking, wif, u, oni]","[ok, lar, jok, wif, u, on]","[ok, lar, ..., joking, wif, u, oni, ...]","[ok, lar, ..., jok, wif, u, on, ...]","[ok, lar, joking, wif, u, oni]","[ok, lar, ..., joking, wif, u, oni, ...]"
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[fre, entry, in, 2, a, wkly, comp, to, win, fa...","[free, entry, in, 2, a, wkly, comp, to, win, f...","[fre, entry, in, 2, a, wkly, comp, to, win, fa...","[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, so, ear, hor, u, c, already, the...","[u, dun, say, so, early, hor, ..., u, c, alrea...","[u, dun, say, so, ear, hor, ..., u, c, already...","[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, so, early, hor, ..., u, c, alrea..."
4,ham,"[nah, i, do, n't, think, he, go, to, usf, he, ...","[nah, i, do, n't, think, he, goe, to, usf, he,...","[nah, i, do, n't, think, he, go, to, usf, ,, h...","[nah, i, do, n't, think, he, goe, to, usf, ,, ...","[nah, i, do, n't, think, he, goes, to, usf, he...","[nah, i, do, n't, think, he, goes, to, usf, ,,..."


Но в таком виде ни один векторайзер ничего не примет (как оказалось), поэтому попробуем сджойнить листы

In [21]:
for col in msg_df.columns[1:]:
    msg_df[col] = msg_df[col].map(lambda x: ' '.join(x))
msg_df.head(2)

,label,nopunctlem,nopunctstem,punctlem,punctstem,tokenized_no_punct,tokenized_punct
0,ham,go until jurong point crazy available only in ...,go until jurong point crazy avail on in bug n ...,"go until jurong point , crazy.. available only...","go until jurong point , crazy.. avail on in bu...",go until jurong point crazy available only in ...,"go until jurong point , crazy.. available only..."
1,ham,ok lar joking wif u oni,ok lar jok wif u on,ok lar ... joking wif u oni ...,ok lar ... jok wif u on ...,ok lar joking wif u oni,ok lar ... joking wif u oni ...


Окей.

In [24]:
# строим пайплайны
CV_clf = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()), 
                   ('clf', MultinomialNB()),])
TI_clf = Pipeline([('vect', TfidfVectorizer()),
                   ('tfidf', TfidfTransformer()), 
                   ('clf', MultinomialNB()),])

In [25]:
# и настраиваем gridsearch
from sklearn.model_selection import GridSearchCV
parameters = {'vect__stop_words': [None, 'english'],
              'vect__min_df': [1, 3],
              'vect__max_df': [1.0, 0.8],}

In [45]:
# теперь будем это всё делить.
# и запускать.

ham = msg_df.loc[msg_df['label']=='ham']
spam = msg_df.loc[msg_df['label']=='spam']


gs_stats = {}
len_ham_sample = len(ham)//6

# 0 колонка - это label, его не перебираем
for column in msg_df.columns[1:]:
    for i in range(6):
        # делим на шесть равных выборок (игнорируем в итоге не больше пяти наблюдений)
        hham = ham[i*len_ham_sample:i*len_ham_sample+len_ham_sample]
        # очень плохо, что они не перемешаны, ноо
        sample = pd.concat([spam, hham])
        X = sample[column]
        y = sample['label']
        gs_cv = GridSearchCV(CV_clf, parameters)
        gs_ti = GridSearchCV(TI_clf, parameters)
        gs_cv = gs_cv.fit(X, y)
        gs_ti = gs_ti.fit(X, y)
        gs_stats[column] = [gs_cv]
        gs_stats[column].append(gs_ti)

In [48]:
gs_stats['punctlem'][0].best_estimator_.get_params()

{'clf': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
 'clf__alpha': 1.0,
 'clf__class_prior': None,
 'clf__fit_prior': True,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
  

In [30]:
len(gs_stats)

4198

In [33]:
column = 'nopunctlem'
stats = []
len_ham_sample = len(ham)//6
for i in range(6):
    # делим на шесть равных выборок (игнорируем в итоге не больше пяти наблюдений)
    hham = ham[i*len_ham_sample:i*len_ham_sample+len_ham_sample]
    print('ham from {} to {}'.format(i*len_ham_sample, i*len_ham_sample+len_ham_sample))
    # очень плохо, что они не перемешаны, ноо
    sample = pd.concat([spam, hham])
    X = sample[column]
    y = sample['label']
    # строим пайплайны
    CV_clf = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()), 
                   ('clf', MultinomialNB()),])
    TI_clf = Pipeline([('vect', TfidfVectorizer()),
                   ('tfidf', TfidfTransformer()), 
                   ('clf', MultinomialNB()),])
    gs_cv = GridSearchCV(CV_clf, parameters)
    gs_ti = GridSearchCV(TI_clf, parameters)
    print('fitting CountVectorizer...')
    gs_cv = gs_cv.fit(X, y)
    print('fitting TfidfVectorizer...')
    gs_ti = gs_ti.fit(X, y)
    stats.append(gs_cv)
    stats.append(gs_ti)

ham from 0 to 804
fitting CountVectorizer...
fitting TfidfVectorizer...
ham from 804 to 1608
fitting CountVectorizer...
fitting TfidfVectorizer...
ham from 1608 to 2412
fitting CountVectorizer...
fitting TfidfVectorizer...
ham from 2412 to 3216
fitting CountVectorizer...
fitting TfidfVectorizer...
ham from 3216 to 4020
fitting CountVectorizer...
fitting TfidfVectorizer...
ham from 4020 to 4824
fitting CountVectorizer...
fitting TfidfVectorizer...


In [41]:
stats[6].best_estimator_.get_params()

{'clf': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
 'clf__alpha': 1.0,
 'clf__class_prior': None,
 'clf__fit_prior': True,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
  

2(+2). Сравнить результаты байесовского классификатора, решающего дерева и RandomForest. Помимо стандартных метрик оценки качества модели, необходимо построить learning curve, ROC-curve, classification report и интерпретировать эти результаты.

3(+2). А что, если в качестве предикторов брать не количество вхождений слов, а конструировать специальные признаки? Прежде всего, необходимо разделить таблицу на training set и test set в соотношении 80:20, test set не открывать до этапа оценки модели. С помощью pandas проверить, отличаются ли перечисленные ниже параметры (иможно придумать другие) для разных классов (spam/ham), и собрать матрицу признаков для обучения. Примеры признаков: длина сообщения, количество букв в ВЕРХНЕМ РЕГИСТРЕ, восклицательных знаков, цифр, запятых, каких-то конкретных слов (для этого можно построить частотный словарь по сообщениям каждого класса). Прокомментировать свой выбор. Векторизовать документы и построить классификатор. Оценить модель на проверочной выборке.

http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
http://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

In [ ]:
messages['length'] = messages['message'].map(lambda text: len(text))

In [ ]:
# print(messages.head())

# tokens = [word_tokenize(msg) for msg in messages]


def tokenize(text):
    text = text.lower()
    return word_tokenize(text)

# messages.message.head().apply(tokenize)
# messages.message = messages.message.apply(tokenize)

# print(messages.head())

In [ ]:
bow = CountVectorizer()
bow.fit_transform(messages['message'])
# print(bow.vocabulary_)

# m = messages['message'][3]
# print([m])
# bowed_m = bow.transform([m])
# print(bowed_m.shape)
# print(bow.get_feature_names()[1054])

bowed_messages = bow.transform(messages['message'])

In [ ]:
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, messages['label'])

In [ ]:
# naive_model.predict()

# msg_train, msg_test, label_train, label_test = train_test_split(messages['message'], messages['label'], test_size=0.2)
# print(len(msg_train), len(msg_test))
cv_results = cross_val_score(naive_model, bowed_messages, messages['label'], cv=10, scoring='accuracy')
print(cv_results.mean(), cv_results.std())

In [ ]:
# pipeline = Pipeline([
#     ('bow', CountVectorizer(analyzer=tokenize)),
#     ('classifier', MultinomialNB()),
# ])
#
# cv_results = cross_val_score(pipeline,
#                              msg_train,
#                              label_train,
#                              cv=10,
#                              scoring='accuracy',
#                              )
# print(cv_results.mean(), cv_results.std())